In [3]:
import os
import fnmatch
from textblob import TextBlob
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import pos_tag,pos_tag_sents
import regex as re
import operator
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.cross_validation import train_test_split  
from sklearn import metrics
from sklearn import svm
from sklearn.grid_search import GridSearchCV
import pickle
from nltk.corpus import stopwords

C:\Users\praga\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\praga\Anaconda2\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Deceptive opinion spam dataset is a corpus consisting of truthful and deceptive hotel reviews of 20 Chicago hotels. The data is described in two papers according to the sentiment of the review. In particular, the article discusses positive sentiment reviews in 1 and negative sentiment reviews in 2.

The corpus contains:

- 400 deceptive negative reviews from Mechanical Turk
- 400 truthful, negative reviews from Expedia, Hotels.com, Orbitz, Priceline, TripAdvisor, and Yelp
- 400 deceptive positive reviews from Mechanical Turk
-  400 truthful, positive reviews from TripAdvisor

In [7]:
path = '/Users/praga/Downloads/op_spam_v1.4/'

label = []

configfiles = [os.path.join(subdir,f)
for subdir, dirs, files in os.walk(path)
    for f in fnmatch.filter(files, '*.txt')]
len(configfiles)

1600

In [8]:
configfiles[0]

'/Users/praga/Downloads/op_spam_v1.4/negative_polarity\\deceptive_from_MTurk\\fold1\\d_hilton_1.txt'

In order to extract the labels we need some kind of filter.

In [9]:
for f in configfiles:
    c = re.search('(trut|deceptiv)\w',f)
    label.append(c.group())

In [15]:
labels = pd.DataFrame(label, columns = ['Labels'])
labels.sample(10)

,Labels
955,deceptive
764,truth
354,deceptive
1163,deceptive
1146,deceptive
33,deceptive
1466,truth
1455,truth
688,truth
1203,truth


It's time to extract the reviews from the text files.

In [16]:
review = []
directory =os.path.join("/Users/praga/Downloads/op_spam_v1.4/")
for subdir,dirs ,files in os.walk(directory):
    print (subdir)
    for file in files:
        if fnmatch.filter(files, '*.txt'):
            f=open(os.path.join(subdir, file),'r')
            a = f.read()
            review.append(a)

/Users/praga/Downloads/op_spam_v1.4/
/Users/praga/Downloads/op_spam_v1.4/negative_polarity
/Users/praga/Downloads/op_spam_v1.4/negative_polarity\deceptive_from_MTurk
/Users/praga/Downloads/op_spam_v1.4/negative_polarity\deceptive_from_MTurk\fold1
/Users/praga/Downloads/op_spam_v1.4/negative_polarity\deceptive_from_MTurk\fold2
/Users/praga/Downloads/op_spam_v1.4/negative_polarity\deceptive_from_MTurk\fold3
/Users/praga/Downloads/op_spam_v1.4/negative_polarity\deceptive_from_MTurk\fold4
/Users/praga/Downloads/op_spam_v1.4/negative_polarity\deceptive_from_MTurk\fold5
/Users/praga/Downloads/op_spam_v1.4/negative_polarity\truthful_from_Web
/Users/praga/Downloads/op_spam_v1.4/negative_polarity\truthful_from_Web\fold1
/Users/praga/Downloads/op_spam_v1.4/negative_polarity\truthful_from_Web\fold2
/Users/praga/Downloads/op_spam_v1.4/negative_polarity\truthful_from_Web\fold3
/Users/praga/Downloads/op_spam_v1.4/negative_polarity\truthful_from_Web\fold4
/Users/praga/Downloads/op_spam_v1.4/negative_

In [18]:
reviews = pd.DataFrame(review, columns = ['HotelReviews'])
reviews.head()

,HotelReviews
0,We stayed at the Schicago Hilton for 4 days an...
1,Hotel is located 1/2 mile from the train stati...
2,I made my reservation at the Hilton Chicago be...
3,"When most people think Hilton, they think luxu..."
4,My husband and I recently stayed stayed at the...


Now merge both the labels and the reviews dataframe.

In [22]:
result = pd.merge(reviews, labels,right_index=True,left_index = True)


result['HotelReviews'] = result['HotelReviews'].map(lambda x: x.lower()) ## lower case
result.sample(10)

,HotelReviews,Labels
282,i recently stayed at the sheraton chicago hote...,deceptive
1106,my family and i recently attended a wedding in...,deceptive
767,after reading all the lukewarm reviews for thi...,truth
342,i recently stayed at the amalfi hotel in chica...,deceptive
375,my wife and i were excited to visit and shop a...,deceptive
175,imagine flying to the windy city for business....,deceptive
1598,stayed here october 31 through november 5 for ...,truth
714,just wanted to let everyone know if you are lo...,truth
1230,i believe i can not describe how amazing was o...,truth
650,this review has two parts. . .i advise reading...,truth


In [29]:
result.to_csv('/Users/praga/Downloads/op_spam_v1.4/df.csv',index=False)